# Abstract for the Project

## Task

The task is to test which model architecture performs best at imdb sentiment classification.

## Model

The models utilize BERT embeddings and various combinations of lstm , cnn , and graph techniques.

## Experiment

The experiments are in progress. We are looking for accuracy, failure points, and interpretability. The goal is to test for full data, limited data, and limited label scenarios.

## Datasets

The main dataset is the IMDB movie reviews and whatever was used to produce the BERT embeddings.

# Code

## Import statements

In [1]:
%pip install -U --quiet tensorflow-text==2.8.1

     |████████████████████████████████| 4.9 MB 5.7 MB/s 


In [2]:
%pip install --quiet neural_structured_learning

     |████████████████████████████████| 120 kB 5.0 MB/s 


In [3]:
%pip install --quiet spektral

     |████████████████████████████████| 129 kB 4.9 MB/s 


In [4]:
import matplotlib.pyplot as plt
import numpy as np
import neural_structured_learning as nsl
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text
import pandas as pd

from spektral.data import Graph
from spektral.data import Dataset
from spektral.transforms import GCNFilter
import pandas as pd
from spektral.utils.sparse import sp_matrix_to_sp_tensor
from spektral.data.loaders import SingleLoader
from spektral.layers import GCNConv
from spektral.models.gcn import GCN
from spektral.transforms import LayerPreprocess
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import RMSprop

# Resets notebook state
tf.keras.backend.clear_session()

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print(
    "GPU is",
    "available" if tf.config.list_physical_devices("GPU") else "NOT AVAILABLE")

Version:  2.8.2
Eager mode:  True
Hub version:  0.12.0
GPU is available


## GCN Model

### Load data

In [5]:
# Load data
# dataset = Citation(data, normalize_x=True, transforms=[LayerPreprocess(GCNConv)])
imdb = tf.keras.datasets.imdb
(pp_train_data, pp_train_labels), (pp_test_data, pp_test_labels) = (
    imdb.load_data(num_words=10000))



17473536/17464789 [==============================] - 0s 0us/step


In [6]:
# This block limits how much is loaded to keep debugging short
# don't run on the final experiments
validation_fraction = 0.2
limit = 25000

pp_validation_data = pp_train_data[0: int(limit * validation_fraction)]
pp_validation_labels = pp_train_labels[0: int(limit * validation_fraction)]
pp_train_data = pp_train_data[int(limit * validation_fraction): limit]
pp_train_labels = pp_train_labels[int(limit * validation_fraction): limit]
pp_test_data = pp_test_data[0:limit]
pp_test_labels = pp_test_labels[0:limit]

In [7]:
print('Training entries: {}, labels: {}'.format(
    len(pp_train_data), len(pp_train_labels)))
training_samples_count = len(pp_train_data)

Training entries: 20000, labels: 20000


In [8]:
print('Validation entries: {}, labels: {}'.format(
    len(pp_validation_data), len(pp_validation_labels)))

Validation entries: 5000, labels: 5000


In [9]:
print('Test entries: {}, labels: {}'.format(
    len(pp_test_data), len(pp_test_labels)))

Test entries: 25000, labels: 25000


In [10]:
print(pp_train_data[0])

[1, 13, 104, 14, 2856, 77, 6, 542, 20, 5, 49, 7, 94, 531, 26, 608, 21, 11, 226, 45, 6, 503, 20, 172, 44, 4, 114, 531, 26, 608, 21, 12, 47, 450, 1512, 5, 2, 15, 152, 179, 763, 56, 116, 9, 669, 608, 207, 110, 433, 7, 14, 99, 10, 10, 4, 454, 732, 4, 651, 19, 1880, 1953, 11, 4, 2092, 231, 646, 21, 484, 1744, 2980, 143, 49, 392, 2, 11, 550, 1270, 646, 40, 4, 9331, 69, 115, 110, 6439, 159, 12, 214, 1076, 128, 39, 50, 21, 24, 76, 10, 10, 48, 335, 2146, 851, 14, 79, 160, 31, 99, 5, 361, 14, 31, 17, 6, 2272, 56, 401, 1398, 364, 5, 12, 238, 157, 18, 25]


In [11]:
len(pp_train_data[0]), len(pp_train_data[1])

(124, 171)

In [12]:
def sampleFunction(inputFeature):
    """
    This is a description of the function
    
    Args:
        inputFeature - (np.ndarray) This is what the feature is
    Returns:
        result - (int) This is what is returned
    """

    result = 55
    return result

def buildReverseWordIndex(dataset):
    """
    Convert the index back to words with proper accounting for 
    the special characters reserved at the beginning of the dictionary

    Args: 
        dataset - (keras.dataset) The dataset to use
    Returns:
        buildReverseWordIndex - (dict) A dictionary mapping words to an integer index
    """
    wordIndex = dataset.get_word_index()

    # The first indices are reserved
    wordIndex = {k: (v + 3) for k, v in wordIndex.items()}
    wordIndex['<PAD>'] = 0
    wordIndex['<START>'] = 1
    wordIndex['<UNK>'] = 2  # unknown
    wordIndex['<UNUSED>'] = 3
    return dict((value, key) for (key, value) in wordIndex.items())

def decodeReview(text, reverseWordIndex):
    """
    Uses build_reverse_word_index to decode original data format into text
    
    Args:
        text - (np.ndarray) The text to decode
        reverseWordIndex - (dict) The reverse word index to use
    Returns:
        decodedReview - (string) The decoded review
    """
    return ' '.join([reverseWordIndex.get(i, '?') for i in text])


In [13]:
reverseWordIndex = buildReverseWordIndex(imdb)
decodeReview(pp_train_data[0], reverseWordIndex)

1654784/1641221 [==============================] - 0s 0us/step


"<START> i think this could've been a decent movie and some of its parts are ok but in whole it's a b movie same about the plot parts are ok but it has several holes and <UNK> that doesn't quite add up acting is mostly ok i've seen worse of this too br br the beginning sets the level with cars driving in the desert making cool but totally unnecessary jumps through some small <UNK> in slow motion cool like the drivers had never seen sand before it gets slightly better from there but not much br br if you're gonna rent this get another one too and use this one as a warm up keep expectations low and it might work for you"

### Generate BERT Embedding

In [14]:
pretrained_embedding = 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/2'

In [15]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
preprocessor = hub.KerasLayer(
    'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')

In [16]:
encoder_inputs = preprocessor(text_input)

encoder = hub.KerasLayer(
    'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/2',
    trainable=True)

outputs = encoder(encoder_inputs)

pooled_output = outputs['pooled_output'] # [batch_size, 128].
# [batch_size, seq_length, 128].

sequence_output = outputs['sequence_output']

In [17]:
def int64Feature(value):
    """
    Returns int64 tf.train.Feature.

    Args:
        value - (np.ndarray) array of ints
    """
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value.tolist()))


def bytesFeature(value):
    """
    Returns bytes tf.train.Feature.

    Args:
        value - (string) string
    """
    return tf.train.Feature(
        bytes_list=tf.train.BytesList(value=[value.encode('utf-8')]))


def floatFeature(value):
    """
    Returns float tf.train.Feature.

    Args:
        value - (np.ndarray) array of floats

    """
    return tf.train.Feature(float_list=tf.train.FloatList(value=value.tolist()))


In [18]:
def createBertEmbeddingExample(wordVector, recordID, reverseWordIndex, encoder, preprocessor):
    """
    Create tf.Example containing the sample's embedding and its ID.
    
    Args:
        wordVector - (np.ndarray) the text to decode
        recordId - (int) ID of the sample
        reverseWordIndex - (dict) The reverse word index to use
        encoder - (string) encoder name
        preprocessor - (string) preprocessor name
    Returns:
        example - (tf.Example) tf.Example containing the sample's embedding and its ID
    """

    text = decodeReview(wordVector, reverseWordIndex)

    # Shape = [batch_size,].
    sentenceEmbedding = encoder(preprocessor(tf.reshape(text, shape=[-1, ])))['pooled_output']
    
    # Flatten the sentence embedding back to 1-D.
    sentenceEmbedding = tf.reshape(sentenceEmbedding, shape=[-1])
    
    features = {
        'id': bytesFeature(str(recordID)),
        'embedding': floatFeature(sentenceEmbedding.numpy())
    }
    return tf.train.Example(features=tf.train.Features(feature=features))


def createBertEmbedding(wordVectors, outputPath, startingRecordId, reverseWordIndex, encoder, preprocessor):
    """
    Create full set of BERT embeddings

    Args:
        wordVectors - (np.ndarray) all text to decode
        outputPath - (string) path to output file
        startingRecordId - (int) ID of the first sample
        reverseWordIndex - (dict) The reverse word index to use
        encoder - (string) encoder name
        preprocessor - (string) preprocessor name
    Returns:
        recordID - (int) ID of the last sample
    """
    recordID = int(startingRecordId)
    with tf.io.TFRecordWriter(outputPath) as writer:
        for word_vector in wordVectors:
            example = createBertEmbeddingExample(word_vector, recordID, reverseWordIndex, encoder, preprocessor)
            recordID = recordID + 1
            writer.write(example.SerializeToString())
    return recordID


In [19]:
!mkdir ./imdb

In [20]:
# Generate full BERT embeddings

bertEmbeddingsPath = './imdb/bertEmeddings.tfr'
createBertEmbedding(pp_train_data, bertEmbeddingsPath, 0, reverseWordIndex, encoder, preprocessor)

bertEmbeddingsPath_validation = './imdb/bertEmeddings_validation.tfr'
createBertEmbedding(pp_validation_data, bertEmbeddingsPath_validation, 0, reverseWordIndex, encoder, preprocessor)

bertEmbeddingsPath_test = './imdb/bertEmeddings_test.tfr'
createBertEmbedding(pp_test_data, bertEmbeddingsPath_test, 0, reverseWordIndex, encoder, preprocessor)
bertEmbeddingsPath

'./imdb/bertEmeddings.tfr'

In [21]:
!wc -l ./imdb/bertEmeddings.tfr

186718 ./imdb/bertEmeddings.tfr


In [22]:
def createBertEmbeddingExample_tf(wordVector, reverseWordIndex, encoder, preprocessor):
    """
    Create tf.Example containing the sample's embedding and its ID.
    
    Args:
        wordVector - (np.ndarray) the text to decode
        recordId - (int) ID of the sample
        reverseWordIndex - (dict) The reverse word index to use
        encoder - (string) encoder name
        preprocessor - (string) preprocessor name
    Returns:
        example - (tf.Example) tf.Example containing the sample's embedding and its ID
    """

    text = decodeReview(wordVector, reverseWordIndex)

    # Shape = [batch_size,].
    sentenceEmbedding = encoder(preprocessor(tf.reshape(text, shape=[-1, ])))['pooled_output']
    
    # Flatten the sentence embedding back to 1-D.
    sentenceEmbedding = tf.reshape(sentenceEmbedding, shape=[-1])
    
    # features = {
    #     'id': bytesFeature(str(recordID)),
    #     'embedding': floatFeature(sentenceEmbedding.numpy())
    # }
    # return tf.train.Example(features=tf.train.Features(feature=features))
    return sentenceEmbedding

def createBertEmbedding_tf(wordVectors, reverseWordIndex, encoder, preprocessor):
    """
    Create full set of BERT embeddings

    Args:
        wordVectors - (np.ndarray) all text to decode
        outputPath - (string) path to output file
        startingRecordId - (int) ID of the first sample
        reverseWordIndex - (dict) The reverse word index to use
        encoder - (string) encoder name
        preprocessor - (string) preprocessor name
    Returns:
        recordID - (int) ID of the last sample
    """
    sentenceEmbeddings = []
    for word_vector in wordVectors:
        example = createBertEmbeddingExample_tf(word_vector, reverseWordIndex, encoder, preprocessor)
        sentenceEmbeddings.append(example)
    sentenceEmbeddings_np = np.array(sentenceEmbeddings)
    return sentenceEmbeddings_np


In [23]:
bertEmbeddings_np = createBertEmbedding_tf(pp_train_data, reverseWordIndex, encoder, preprocessor)
bertEmbeddings_np

bertEmbeddings_va_np = createBertEmbedding_tf(pp_validation_data, reverseWordIndex, encoder, preprocessor)
bertEmbeddings_te_np = createBertEmbedding_tf(pp_test_data, reverseWordIndex, encoder, preprocessor)


In [24]:
np.shape(bertEmbeddings_np)

(20000, 128)

### Construct Graph Using BERT Embedding

In [25]:
similarity_threshold = 0.99

In [26]:

graph_builder_config = nsl.configs.GraphBuilderConfig(
    similarity_threshold=similarity_threshold, lsh_splits=32, lsh_rounds=15, random_seed=42)
nsl.tools.build_graph_from_config(['./imdb/bertEmeddings.tfr'],
                                  './imdb/graph_99.tsv',
                                  graph_builder_config)

In [27]:
graph_builder_config = nsl.configs.GraphBuilderConfig(
    similarity_threshold=similarity_threshold, lsh_splits=32, lsh_rounds=15, random_seed=42)
nsl.tools.build_graph_from_config(['./imdb/bertEmeddings_test.tfr'],
                                  './imdb/graph_99_test.tsv',
                                  graph_builder_config)

In [28]:
graph_builder_config = nsl.configs.GraphBuilderConfig(
    similarity_threshold=similarity_threshold, lsh_splits=32, lsh_rounds=15, random_seed=42)
nsl.tools.build_graph_from_config(['./imdb/bertEmeddings_validation.tfr'],
                                  './imdb/graph_99_validation.tsv',
                                  graph_builder_config)

In [29]:
!wc -l ./imdb/graph_99.tsv

24066 ./imdb/graph_99.tsv


### Convert NSL Graph to Spektral Graph



In [30]:
df = pd.read_csv('./imdb/graph_99.tsv', sep="\t") 
imdb_graph_a_adjacency_matrix = df.values
size = len(pp_train_data)
imdb_graph_a = np.zeros((size, size))
for row in imdb_graph_a_adjacency_matrix:
  imdb_graph_a[int(row[0]),int(row[1])] = row[2]


In [31]:
df = pd.read_csv('./imdb/graph_99_validation.tsv', sep="\t") 
imdb_graph_va_a_adjacency_matrix = df.values
size = len(pp_validation_data)
imdb_graph_va_a = np.zeros((size, size))
for row in imdb_graph_va_a_adjacency_matrix:
  imdb_graph_va_a[int(row[0]),int(row[1])] = row[2]


In [32]:
df = pd.read_csv('./imdb/graph_99_test.tsv', sep="\t") 
imdb_graph_te_a_adjacency_matrix = df.values
size = len(pp_test_data)
imdb_graph_te_a = np.zeros((size, size))
for row in imdb_graph_te_a_adjacency_matrix:
  imdb_graph_te_a[int(row[0]),int(row[1])] = row[2]


In [33]:
max_seq_length_slice = 256
# pp_train_data_sliced = np.zeros((len(pp_train_data), max_seq_length_slice))
# i = 0
# while i < len(pp_train_data):
#   if len(pp_train_data[i]) > max_seq_length_slice:
#     pp_train_data_sliced[i] = pp_train_data[i][:max_seq_length_slice]
#   else:
#     pp_train_data_sliced[i] = np.append(np.array(pp_train_data[i]), np.zeros(max_seq_length_slice - len(pp_train_data[i])))
#   i = i + 1




In [34]:
print(len(bertEmbeddings_np),len(bertEmbeddings_np[0]))
print(bertEmbeddings_np.shape)

20000 128
(20000, 128)


In [35]:
pp_train_labels_binary = np.zeros((len(pp_train_labels),2))
j = 0
for _ in pp_train_labels:
  if _ == 1:
    pp_train_labels_binary[j][1] = 1
  else:
    pp_train_labels_binary[j][0] = 1
  j = j + 1


In [36]:
len(pp_validation_labels)

5000

In [37]:
pp_validation_labels_binary = np.zeros((len(pp_validation_labels),2))
j = 0
for _ in pp_validation_labels:
  if _ == 1:
    pp_validation_labels_binary[j][1] = 1
  else:
    pp_validation_labels_binary[j][0] = 1
  j = j + 1


In [38]:
pp_test_labels_binary = np.zeros((len(pp_test_labels),2))
j = 0
for _ in pp_test_labels:
  if _ == 1:
    pp_test_labels_binary[j][1] = 1
  else:
    pp_test_labels_binary[j][0] = 1
  j = j + 1


In [39]:
imdb_graph=Graph(a=imdb_graph_a, x=bertEmbeddings_np, y=pp_train_labels_binary)
imdb_graph_va=Graph(a=imdb_graph_va_a, x=bertEmbeddings_va_np, y=pp_validation_labels_binary)
imdb_graph_te=Graph(a=imdb_graph_te_a, x=bertEmbeddings_te_np, y=pp_test_labels_binary)



In [40]:
class MyDataset(Dataset):
    """
    A dataset of five random graphs.
    """
    def __init__(self, graph, **kwargs):
        # self.nodes = nodes
        # self.feats = feats
        self.graph = graph
        super().__init__(**kwargs)

    def download(self):
        # data = ...  # Download from somewhere
        path = './imdb'
        # Create the directory
        
        # os.mkdir(path)
        # filename = os.path.join(path, 'imdb_graph')
        # np.savez(filename, x=imdb_graph.x, a=imdb_graph.a, y=imdb_graph.y)

    def read(self):
        # We must return a list of Graph objects
        output = []

        # for i in range(5):
        #     data = np.load(os.path.join(self.path, f'graph_{i}.npz'))
        #     output.append(
        #         Graph(x=data['x'], a=data['a'], y=data['y'])
        #     )
        output.append(self.graph)
        return output           

In [ ]:
dataset = MyDataset(imdb_graph)
dataset[0]
dataset.apply(GCNFilter())

dataset_va = MyDataset(imdb_graph_va)
dataset_va.apply(GCNFilter())

dataset_te = MyDataset(imdb_graph_te)
dataset_te.apply(GCNFilter())

In [ ]:
dataset[0].x.shape
dataset_va[0].x.shape
dataset_te[0].x.shape

### Train the model

In [ ]:
from tensorflow.keras.optimizers import SGD
learning_rate = 1e-3
seed = 0
epochs = 100
patience = 10

tf.random.set_seed(seed=seed)  # make weight initialization reproducible

In [ ]:
model = GCN(n_labels=imdb_graph.n_labels)

In [ ]:
model.compile(
    optimizer=Adam(learning_rate),
    loss=tf.keras.losses.BinaryCrossentropy(),
    weighted_metrics=["acc"],
)

In [ ]:
# Train model
loader_tr = SingleLoader(dataset)
loader_va = SingleLoader(dataset_va)
model.fit(
    loader_tr.load(),
    steps_per_epoch=loader_tr.steps_per_epoch,
    validation_data = loader_va.load(),
    validation_steps = loader_va.steps_per_epoch,
    epochs=epochs,
    # callbacks=[EarlyStopping(patience=patience, restore_best_weights=True)],
)

### Evaluate Model

In [ ]:
# Evaluate model
print("Evaluating model.")
loader_te = SingleLoader(dataset_te)
eval_results = model.evaluate(loader_te.load(), steps=loader_te.steps_per_epoch)
print("Done.\n" "Test loss: {}\n" "Test accuracy: {}".format(*eval_results))